In [ ]:



#preparing datasets
!pip install transformers datasets[sentencepiece]
!pip install transformers
!pip install torch==1.8.0 torchvision==0.9.0 torchaudio==0.8.0

from torch.utils.data import DataLoader
from datasets import load_dataset,  load_metric
from torch.utils.data import DataLoader
from transformers import AutoTokenizer, DataCollatorWithPadding, AutoModelForSequenceClassification

raw_datasets = load_dataset("imdb")

#The raw_datasets object is a dictionary with three keys:
# "train", "test" and "unsupervised"

#for our purpuse we will use BERT
tokenizer = AutoTokenizer.from_pretrained("bert-base-cased")

def tk_function(examples):
  return tokenizer(examples["text"], padding="max_length", truncation = True)

tokenized_datasets = raw_datasets.map(tk_function, batched=True)

#before using pytorch, we need to modify our tokenized datasets according to Pytorch requirements
#1. we remove the column "text" (column corresponding to values the model does not expect) - we only keep the tokenized values
#2. we rename the "label" column to "labels"
#3. change the format of the datasets from lists to TENSORS

tokenized_datasets = tokenized_datasets.remove_columns(["text"])
tokenized_datasets = tokenized_datasets.rename_column("label", "labels")
tokenized_datasets.set_format("torch")





#Data collators are objects that will form a batch by using a list of dataset elements as input. These elements are of the same type as the elements of train_dataset or eval_dataset.
data_collator = DataCollatorWithPadding(tokenizer)

train_dataloader = DataLoader(
  tokenized_datasets["train"], shuffle=True, batch_size=8, collate_fn=data_collator
)
eval_dataloader = DataLoader(
  tokenized_datasets["test"], batch_size=8, collate_fn=data_collator
)

#DISCLAIMER !!!! - I did not use the below code from the tutorial, because it reporeted different errors ('label', ...)
#Instead I used the above way of declaring train & eval dataloaders
#small_train_dataset = tokenized_datasets["train"].shuffle(seed=42).select(range(1000))
#small_eval_dataset = tokenized_datasets["test"].shuffle(seed=42).select(range(1000))
#now we can define our dataloader class. It represents a Python iterable over a dataset, 
#with support for map-style and iterable-style datasets, customizing data loading order,
#automatic batching, single- and multi-process data loading, automatic memory pinning.
#train_dataloader = DataLoader(small_train_dataset, shuffle=True, batch_size=8)
#eval_dataloader = DataLoader(small_eval_dataset, batch_size=8)

# again we define our model

model = AutoModelForSequenceClassification.from_pretrained("bert-base-cased", num_labels=2)

# now the only two things missing are an optimizer and a learning rate scheduler
# AdamW is a default optimizer used by Traniner

from transformers import AdamW, get_scheduler

optimizer = AdamW(model.parameters(), lr=5e-5)

#Is this necessary here?:
  #loss = outputs.loss
  #loss.backward()
  #optimizer.step()
  #optimizer.zero_grad()

num_epochs = 3
num_training_steps = num_epochs * len(train_dataloader)
lr_scheduler = get_scheduler(
    "linear",
    optimizer=optimizer,
    num_warmup_steps=0,
    num_training_steps=num_training_steps
)


#to use our GPU to train our model we define a device as follows:

import torch

device = torch.device("cuda") if torch.cuda.is_available() else torch.device("cpu")
model.to(device)

# we can use tqdm library to display a progress bar

from tqdm.auto import tqdm

progress_bar = tqdm(range(num_training_steps))

model.train()
for epoch in range(num_epochs):
    for batch in train_dataloader:
        batch = {k: v.to(device) for k, v in batch.items()}
        outputs = model(**batch)
        loss = outputs.loss
        loss.backward()
        optimizer.step()
        lr_scheduler.step()
        optimizer.zero_grad()
        progress_bar.update(1)

#to check the results we just use a metric from datasets library

metric= load_metric("accuracy")
model.eval()
for batch in eval_dataloader:
    batch = {k: v.to(device) for k, v in batch.items()}
    with torch.no_grad():
        outputs = model(**batch)

    logits = outputs.logits
    predictions = torch.argmax(logits, dim=-1)
    metric.add_batch(predictions=predictions, references=batch["labels"])

metric.compute()




     |████████████████████████████████| 2.6 MB 11.4 MB/s 
     |████████████████████████████████| 264 kB 27.0 MB/s 
     |████████████████████████████████| 895 kB 42.7 MB/s 
     |████████████████████████████████| 3.3 MB 41.3 MB/s 
     |████████████████████████████████| 636 kB 50.2 MB/s 
     |████████████████████████████████| 76 kB 4.8 MB/s 
     |████████████████████████████████| 118 kB 54.4 MB/s 
     |████████████████████████████████| 243 kB 49.1 MB/s 
  Attempting uninstall: tqdm
    Found existing installation: tqdm 4.41.1
    Uninstalling tqdm-4.41.1:
      Successfully uninstalled tqdm-4.41.1
  Attempting uninstall: pyyaml
    Found existing installation: PyYAML 3.13
    Uninstalling PyYAML-3.13:
      Successfully uninstalled PyYAML-3.13
     |████████████████████████████████| 735.5 MB 11 kB/s 
     |████████████████████████████████| 17.3 MB 23 kB/s 
     |████████████████████████████████| 1.9 MB 39.1 MB/s 
  Attempting uninstall: torch
    Found existing installation: torch 

Downloading:   0%|          | 0.00/1.92k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/1.05k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/84.1M [00:00<?, ?B/s]

0 examples [00:00, ? examples/s]

...

  0%|          | 0/25 [00:00<?, ?ba/s]

  0%|          | 0/50 [00:00<?, ?ba/s]

Downloading:   0%|          | 0.00/436M [00:00<?, ?B/s]

Some weights of the model checkpoint at bert-base-cased were not used when initializing BertForSequenceClassification: ['cls.predictions.transform.LayerNorm.weight', 'cls.predictions.transform.dense.weight', 'cls.seq_relationship.bias', 'cls.predictions.transform.LayerNorm.bias', 'cls.predictions.transform.dense.bias', 'cls.seq_relationship.weight', 'cls.predictions.decoder.weight', 'cls.predictions.bias']
- This IS expected if you are initializing BertForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of BertForSequenceClassification were not initialized from the model checkpoint at b

  0%|          | 0/9375 [00:00<?, ?it/s]